In [ ]:
%load_ext autoreload
%autoreload 2

import functools
import os
import cv2
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import PIL
import sys
import glob
import subprocess
import pandas as pd
sys.path.append('src/py')

import ai_utils
from image_utils import load_image, show_image, save_image
import image_utils as iu
import time

print("TF Version: ", tf.__version__)
print("TF Hub version: ", hub.__version__)
print("Eager mode enabled: ", tf.executing_eagerly())
print("GPU available: ", tf.config.list_physical_devices('GPU'))

In [ ]:
base_path = "/Users/iman/github/avignon/"    
project_dir =  "projects/graphite/"
input_dir = os.path.join(base_path, project_dir, "graphite_style")
mask_dir = os.path.join(base_path, project_dir, "graphite_original")
output_dir = os.path.join(base_path, project_dir, "output")
styles_dir = os.path.join(base_path, project_dir, "styles")
temp_dir = os.path.join(base_path, project_dir, "temp")

In [ ]:
input_files = glob.glob(os.path.join(mask_dir, '*'))
input_files.sort()

content_img_size = (1920, 1080)
style_img_size = (1920, 1080)

In [ ]:
def run_ebsynth(style, input_file, guide_1, out_dir, uniformity=1,patch_size=5, pyramid_levels=1, search_vote_iters=1, patch_match_iters=1, guide_1_weight=100, guide_2_weight=0.01, guide_2=None):
    output_file = os.path.splitext(os.path.basename(input_file))[0]
    if guide_2:
        command = f"ebsynth -style {style} \
                -guide {guide_1}  {input_file} -weight {guide_1_weight} \
                -guide {guide_2} {input_file} -weight {guide_2_weight} \
                -output {out_dir}/{output_file}.png \
                -uniformity {uniformity} -patchsize {patch_size} -pyramidlevels {pyramid_levels} -searchvoteiters {search_vote_iters} -patchmatchiters {patch_match_iters} "
    else:
        command = f"ebsynth -style {style} \
                -guide {guide_1}  {input_file} -weight {guide_1_weight} \
                -output {out_dir}/{output_file}.png \
                -uniformity {uniformity} -patchsize {patch_size} -pyramidlevels {pyramid_levels} -searchvoteiters {search_vote_iters} -patchmatchiters {patch_match_iters} "

    subprocess.call(command, shell=True)
    return f"{out_dir}/{output_file}.png"

In [ ]:
def translate_data(csv_path, sep="|"):
    data_df = df = pd.read_csv(csv_path, sep=sep,)
    ratios = []
    for number in data_df["numbers"]:
        if '%' in number:
            value = number.replace("%", "")
            ratio = float(value)/100
        elif 'jours' in number:
            ratio = 1.0
        else:
            value = int(number)
            length = len(str(value))
            max_length = int('9' * length)
            ratio = value/max_length
        ratios.append(ratio)
    data_df["ratio"] = ratios
    return data_df

In [ ]:
collected_csv_path = os.path.join(base_path, "questions/q_and_a_truth.csv")
collected_df = translate_data(collected_csv_path)

In [ ]:
collected_df

In [ ]:
static_csv_path = os.path.join(base_path, "questions/static_q_and_a.csv")
static_df = translate_data(static_csv_path)

In [ ]:
question_segments = 12

In [ ]:
def get_coords(digit, rows, cols):
    position = digit % (rows*cols) 
    row = position // cols 
    col = position % cols 
    return row, col

In [ ]:
def put_text_wrap(image, text, org, font, font_scale, color, thickness, line_spacing, max_width):
    words = text.split()  # Split text into individual words
    lines = []
    current_line = ""

    for word in words:
        # Check if adding the current word exceeds the maximum width
        if cv2.getTextSize(current_line + " " + word, font, font_scale, thickness)[0][0] <= max_width:
            current_line += " " + word
        else:
            lines.append(current_line.strip())
            current_line = word

    lines.append(current_line.strip())  # Add the last line

    # Draw each line of text with appropriate y-coordinates
    for i, line in enumerate(lines):
        y = org[1] + (i * line_spacing)
        cv2.putText(image, line, (org[0], y), font, font_scale, color, thickness)

In [ ]:
def get_style(input_file, out_dir, segments, static_df, collected_df, grid_rows=6, grid_cols=2, fsize_max=17, fsize_min=7, fstroke_max=12, colour_max=104):
    input_image = load_image(input_file)
    
    height = input_image.shape[0]
    width = input_image.shape[1]

    blank_image = np.zeros((height, width, 3), np.uint8)
    blank_image.fill(0)

    cell_width = width // 2
    cell_height = height // 6
    
    joined_df = pd.concat([collected_df, static_df]) 


    for segment in segments:
        seg_joined_df = joined_df[joined_df['question number'] == segment]
        i_row, i_col = get_coords(segment, rows=grid_rows, cols=grid_cols)
    
        x = i_col * cell_width
        y = i_row * cell_height - int(cell_height/4)

        adj_x = x
        adj_y = y + cell_height
        
        for index, row in seg_joined_df.iterrows():
            text = row['question']
            ratio = row['ratio']
            put_text_wrap(blank_image,text, (adj_x, adj_y),  cv2.FONT_HERSHEY_SIMPLEX, max(12 * ratio, 4), (10, 5, 10), 6, 40, 80)
            # cv2.putText(blank_image, text, (adj_x, adj_y), cv2.FONT_HERSHEY_SIMPLEX, fsize_max*1, (10, 10, 10), int(fstroke_max*1))

        for index, row in seg_joined_df.iterrows():
            text = row['numbers']
            ratio = row['ratio']
            cv2.putText(blank_image, text, (adj_x, adj_y), cv2.FONT_HERSHEY_SIMPLEX, max(fsize_max*ratio, fsize_min), (255*ratio, colour_max*1, 10), int(fstroke_max*3))
            

    temp_file_path = os.path.join(out_dir, os.path.basename(input_file))

    s_image = iu.overlay(blank_image,input_image)
    s_image = iu.edges(s_image)
    s_image = cv2.cvtColor(s_image, cv2.COLOR_GRAY2BGR)
    s_image = iu.overlay(input_image, s_image, 0.5)
    s_image = iu.overlay(blank_image, s_image, 0.1)
    s_image = iu.increase_brightness_contrast(s_image, brightness=50, contrast=40)
    show_image(s_image, fig_size=(10,5))
    save_image(temp_file_path, s_image)
    return temp_file_path

In [ ]:
# get_style(input_file=input_files[4], segments=8, out_dir=temp_dir,static_df=static_df, collected_df=collected_df)

In [ ]:
output_frames = []
frame_rate = 30
segments = []
base_style_path = input_files[0]
for i in range(0,question_segments):
    gen_style = False
    for j in range(i*frame_rate,(i*frame_rate)+frame_rate):
        segments.append(i)
        file_idx = j
        input_file = input_files[file_idx]
        temp_file = os.path.join(temp_dir, os.path.basename(input_files[file_idx]))
        mask_file = input_files[file_idx]
        out_dir = output_dir


        temp_file = get_style(input_file=input_file, out_dir=temp_dir, segments=segments, static_df=static_df, 
        collected_df=collected_df)

        if not gen_style:
            style_file = temp_file
            gen_style = True

        guide_1_weight = 100000
        guide_2 = mask_file
        guide_2_weight = 1000

        start_time = time.time()
        output_frames.append(run_ebsynth(style=style_file, guide_2=guide_2, guide_1_weight=guide_1_weight, guide_2_weight=guide_2_weight, input_file=temp_file, guide_1=mask_file, out_dir=out_dir))
        elapsed_time = time.time() - start_time
        print("Elapsed time: {:.2f} seconds".format(elapsed_time))

        show_image(load_image(output_frames[j]), fig_size=(2.5,5))


In [ ]:
# width = 1080
# height = 1920
# temp_files = []
# for i in range(0,10):
#     blank_image = np.zeros((height, width, 3), np.uint8)
#     blank_image.fill(255)
#     test_i = load_image(input_files[i])

#     grid_rows = 8
#     grid_cols = 4

#     cell_width = test_i.shape[1] // 4
#     cell_height = test_i.shape[0] // 8

#     for row in range(grid_rows):
#         for col in range(grid_cols):
#             # Calculate the position of the cell
#             x = col * cell_width
#             y = row * cell_height

#             # Place the text in the cell
#             text = f'(80, 90)'
#             cv2.putText(blank_image, text, (x+cell_width//2, y+cell_height//2), cv2.FONT_HERSHEY_SIMPLEX, 10, (255, 0, 0), 5)
#     temp_file_path = os.path.join(temp_dir, os.path.basename(input_files[i]))
#     # save_image(temp_file_path, test_i)
#     temp_files.append(temp_file_path)
#     s_image = iu.overlay(blank_image,test_i)
#     s_image = iu.edges(s_image)
#     s_image = cv2.cvtColor(s_image, cv2.COLOR_GRAY2BGR)
#     s_image = iu.overlay(test_i, s_image, 0.5)
#     s_image = iu.overlay(blank_image, s_image, 0.5)
#     show_image(s_image, fig_size=(10,5))
#     # break
#     # save_image(temp_file_path, cv2.cvtColor(s_image, cv2.COLOR_BGR2GRAY))
#     save_image(temp_file_path, s_image)


In [ ]:
# output_frames = []
# for i in range(0,10):
#     input_file = temp_files[i]
#     mask_file = os.path.join(mask_dir, os.path.basename(input_file))
#     out_dir = output_dir
#     # style = os.path.join(base_path,"projects/graphite/temp/0010.png")
#     # style = os.path.join(base_path,"other/text_out2.jpg")
#     style = os.path.join(temp_dir, "0000.png")

#     # guide_2 = os.path.join(base_path,"other/text_out.jpg")
#     guide_2 = None
#     guide_1_weight = 1000
#     guide_2_weight = 1
#     start_time = time.time()
#     output_frames.append(run_ebsynth(style=style, guide_2=guide_2, guide_1_weight=guide_1_weight, guide_2_weight=guide_2_weight, input_file=input_file, guide_1=mask_file, out_dir=out_dir))
#     elapsed_time = time.time() - start_time
#     print("Elapsed time: {:.2f} seconds".format(elapsed_time))

#     show_image(load_image(output_frames[i]), fig_size=(2.5,5))


In [ ]:
# for i, x in enumerate(output_frames):
#     show_image(load_image(output_frames[i]), fig_size=(2.5,5))